Mike Honey: Cloned from [this notebook](https://www.kaggle.com/danielwolffram/cord-19-create-dataframe).

Crashing with memory errors, so breaking into separate notebooks.  This one will just prepare the pmc_json input data and write out the results.

# Load Packages

In [ ]:
import numpy as np 
import pandas as pd

import glob
import json

root_path = '/kaggle/input/CORD-19-research-challenge/'

# Load and Prepare Data

To read the JSON files we follow [COVID EDA: Initial Exploration Tool](https://www.kaggle.com/ivanegapratama/covid-eda-initial-exploration-tool).

# pmc_json

In [ ]:
all_json_pmc = glob.glob(f'{root_path}/**/pmc_json/*.json', recursive=True)
len(all_json_pmc)

This only contains the full text - no abstracts!

In [ ]:
methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']

In [ ]:
[''.join(x.lower() for x in m if x.isalpha()) for m in methods]

In [ ]:
for m in methods:
    print(''.join(x.lower() for x in m if x.isalpha()))

In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.body_text = []
            self.methods = []
            self.results = []

            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            # Methods
            methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha()) #remove numbers and spaces
                if any(m in section_title for m in [''.join(x.lower() for x in m if x.isalpha()) for m in methods]) : 
                    self.methods.append(entry['text'])
            # Results
            results_synonyms = ['result']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha())
                if any(r in section_title for r in results_synonyms) :
                    self.results.append(entry['text'])
                    
            self.body_text = '\n'.join(self.body_text)
            self.methods = '\n'.join(self.methods)
            self.results = '\n'.join(self.results)

    def __repr__(self):
        return f'{self.paper_id}: {self.body_text[:200]}...'
first_row = FileReader(all_json_pmc[0])
print(first_row)

In [ ]:
dict_ = {'paper_id': [], 'body_text': [], 'methods': [], 'results': []}
for idx, entry in enumerate(all_json_pmc):
    if idx % (len(all_json_pmc) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json_pmc)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    dict_['methods'].append(content.methods)
    dict_['results'].append(content.results)
del all_json_pmc

In [ ]:
pmc_text = pd.DataFrame(dict_, columns=['paper_id', 'body_text', 'methods', 'results'])
del dict_
gc.collect()
pmc_text.head()

In [ ]:
root_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_path = f'{root_path}/metadata.csv'
metadata_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
metadata_df.head(2)

In [ ]:
# running out of memory, so filter to papers published from 1 Jan 2020 onwards
metadata_df = metadata_df.drop(metadata_df[metadata_df.publish_time < '2020'].index) 
metadata_df.head(2)

In [ ]:
# only using metadata_df as a filter, so drop non-key columns
metadata_df = metadata_df[['sha']]

In [ ]:
# how = right to drop papers without metadata
pmc_text = pd.merge(pmc_text, metadata_df, left_on='paper_id', right_on='sha', how='right').drop('sha', axis=1)
del df
gc.collect()

In [ ]:
pmc_text.to_csv('pmc_json_df.csv', index=False)